In [1]:
#To allocate memorey for gpu(here we are using 50%)
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [21]:
from keras.models import Sequential
from keras.layers import Dense,Dropout,Flatten
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator,load_img
from keras.utils.np_utils import to_categorical
from keras.layers.convolutional import Conv2D, MaxPooling2D

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import pickle
import cv2
import random

In [5]:
#Parameters
path = "myData/"
label = "labels.csv"
batch_size_val = 32
steps_per_epoch_val = 2000
epoch_val = 10
imageDimension = (32,32,3)
testratio = 0.2
validationratio = 0.2

In [6]:
#importing images
count = 0
images = []
noofclass = []
mylist = os.listdir(path)
print("Total no of classes  detected:",len(mylist))
#print(":",len())

Total no of classes  detected: 43


In [7]:
NoClass = len(mylist)
for x in range(0,len(mylist)):
    myPicList = os.listdir(path+"/"+str(count))
    for y in myPicList:
        curImg = cv2.imread(path+"/"+str(count)+"/"+y)
        images.append(curImg)
        noofclass.append(count)
    count+=1    

In [8]:
images = np.array(images)
noofclass = np.array(noofclass)

In [15]:
print(images)

[[[[129 138 152]
   [131 138 151]
   [131 137 151]
   ...
   [153 127 119]
   [131 116 111]
   [132 124 123]]

  [[128 138 160]
   [134 142 163]
   [134 140 164]
   ...
   [140 128 131]
   [137 128 129]
   [133 127 127]]

  [[146 156 172]
   [147 154 169]
   [151 155 175]
   ...
   [127 125 127]
   [128 126 126]
   [130 126 126]]

  ...

  [[ 93  94  98]
   [ 93  94  99]
   [ 93  95  98]
   ...
   [101 104 102]
   [104 106 106]
   [100 102 102]]

  [[ 91  94  89]
   [ 92  95  89]
   [ 91  94  93]
   ...
   [ 98 102 101]
   [100 106 106]
   [ 96 101 101]]

  [[101 107  91]
   [124 128 107]
   [100 101  94]
   ...
   [ 97 101 102]
   [100 105 105]
   [ 97 101 100]]]


 [[[120 125 129]
   [121 122 127]
   [123 123 128]
   ...
   [133 132 135]
   [118 119 118]
   [116 118 119]]

  [[124 127 132]
   [124 126 130]
   [122 124 130]
   ...
   [143 143 150]
   [126 124 125]
   [122 118 119]]

  [[130 131 143]
   [127 131 145]
   [123 128 143]
   ...
   [171 168 175]
   [155 149 155]
   [150 138

In [16]:
print(noofclass)

[ 0  0  0 ... 42 42 42]


In [9]:
#splitting the data
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(images,noofclass,test_size=0.2)

In [10]:
Xtrain,Xval,Ytrain,Yval = train_test_split(xtrain,ytrain,test_size=0.2)

In [11]:
#read csv files
data = pd.read_csv('labels.csv')
data.head()

,ClassId,Name
0,0,Speed limit (20km/h)
1,1,Speed limit (30km/h)
2,2,Speed limit (50km/h)
3,3,Speed limit (60km/h)
4,4,Speed limit (70km/h)


In [20]:
data.shape

(43, 2)

In [12]:
#preprocessing the image
def preprocess(img):
    img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    img = cv2.equalizeHist(img)
    img = img/255
    return img
Xtrain = np.array(list(map(preprocess,Xtrain)))
Xval = np.array(list(map(preprocess,Xval)))
xtest = np.array(list(map(preprocess,xtest)))

In [13]:
Xtrain = Xtrain.reshape(Xtrain.shape[0],Xtrain.shape[1],Xtrain.shape[2],1)
Xval   = Xval.reshape(Xval.shape[0],Xval.shape[1],Xval.shape[2],1)
xtest  = xtest.reshape(xtest.shape[0],xtest.shape[1],xtest.shape[2],1)

In [15]:
#Data Augmentation
dataGen = ImageDataGenerator(width_shift_range=0.1,height_shift_range=0.1,zoom_range=0.2,shear_range=0.1,rotation_range=10)
dataGen.fit(Xtrain)

In [17]:
batches = dataGen.flow(Xtrain,Ytrain,batch_size=20)
xbatch,ybatch = next(batches)

In [18]:
Ytrain = to_categorical(Ytrain,NoClass)
Yval = to_categorical(Yval,NoClass)
ytest = to_categorical(ytest,NoClass)

In [27]:
batch_size_val=50  # how many to process together
steps_per_epoch_val=2000
epochs_val=10
imageDimesions = (32,32,3)

In [30]:
def myModel():
    no_Of_Filters=60
    size_of_Filter=(5,5) # THIS IS THE KERNEL THAT MOVE AROUND THE IMAGE TO GET THE FEATURES.
                         # THIS WOULD REMOVE 2 PIXELS FROM EACH BORDER WHEN USING 32 32 IMAGE
    size_of_Filter2=(3,3)
    size_of_pool=(2,2)  # SCALE DOWN ALL FEATURE MAP TO GERNALIZE MORE, TO REDUCE OVERFITTING
    no_Of_Nodes = 500   # NO. OF NODES IN HIDDEN LAYERS
    model= Sequential()
    model.add((Conv2D(no_Of_Filters,size_of_Filter,input_shape=(imageDimesions[0],imageDimesions[1],1),activation='relu')))  # ADDING MORE CONVOLUTION LAYERS = LESS FEATURES BUT CAN CAUSE ACCURACY TO INCREASE
    model.add((Conv2D(no_Of_Filters, size_of_Filter, activation='relu')))
    model.add(MaxPooling2D(pool_size=size_of_pool)) # DOES NOT EFFECT THE DEPTH/NO OF FILTERS
 
    model.add((Conv2D(no_Of_Filters//2, size_of_Filter2,activation='relu')))
    model.add((Conv2D(no_Of_Filters // 2, size_of_Filter2, activation='relu')))
    model.add(MaxPooling2D(pool_size=size_of_pool))
    model.add(Dropout(0.5))
 
    model.add(Flatten())
    model.add(Dense(no_Of_Nodes,activation='relu'))
    model.add(Dropout(0.5)) # INPUTS NODES TO DROP WITH EACH UPDATE 1 ALL 0 NONE
    model.add(Dense(NoClass,activation='softmax')) # OUTPUT LAYER
    # COMPILE MODEL
    model.compile(Adam(lr=0.001),loss='categorical_crossentropy',metrics=['accuracy'])
    return model
 
 
############################### TRAIN
model = myModel()
print(model.summary())
history=model.fit_generator(dataGen.flow(Xtrain,Ytrain,batch_size=batch_size_val),steps_per_epoch=steps_per_epoch_val,epochs=epochs_val,validation_data=(Xval,Yval),shuffle=1)

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 28, 28, 60)        1560      
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 24, 24, 60)        90060     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 12, 12, 60)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 10, 10, 30)        16230     
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 8, 8, 30)          8130      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 4, 4, 30)          0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 4, 4, 30)         

In [31]:
from keras.models import load_model
model.save('traffic_sign.h5')